<a href="https://colab.research.google.com/github/nehapokharel/brain-tumor-detection/blob/main/tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing necessary libraries
import os
import skimage
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from skimage import transform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [2]:
#making content of google drive accessible
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!ls "gdrive/MyDrive/MRI_BRAIN_TUMOR/dataset"

no  yes


In [4]:
classes = ["yes", "no"]
n_classes = len(classes)

In [18]:
def load_data():
    """This function loads the data and prepares it to be 
    used by th neneural network. It consists of:
        1. loading images
        2. `normalization`. Original pixel values are in range [0, 255], 
           but neural networks usually work better if the input values are in
           range [0, 1].
        3. resize the image """  
    x = []
    y = []
    path = "gdrive/MyDrive/MRI_BRAIN_TUMOR/dataset"
    for i, c in enumerate(classes):
        for file in os.listdir(os.path.join(path, c)):
            if file.endswith(".jpeg") or file.endswith(".jpg"):
                img = plt.imread(os.path.join(path, c, file))
                img = img / 255.   
                img = transform.resize(img, (32, 32), mode='constant')
                img = img.astype('float32') 
                if len(img.shape) == 3 and img.shape[-1] == 3:
                    x.append(img)
                    y.append(i)  
    x = np.stack(x, axis=0)
    y = np.array(y)
    print(f"x shape: {x.shape}, y shape: {y.shape}")
    return x, y

In [19]:
x, y = load_data() #function call

x shape: (142, 32, 32, 3), y shape: (142,)


In [21]:
#Splitting the dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)

# **Converting lists to numpy array**

In [22]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [23]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [24]:
X_train.shape

(113, 32, 32, 3)

In [25]:
# Create the model

model = keras.Sequential()

model.add(keras.layers.Conv2D(filters=16,kernel_size=9, padding='same', activation='relu', input_shape=(32, 32,3))) 
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.45))

model.add(keras.layers.Conv2D(filters=16,kernel_size=9,padding='same', activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(filters=36, kernel_size=9, padding='same', activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.15))


model.add(keras.layers.Dense(1, activation='sigmoid'))

# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 32, 32, 16)        3904      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 16)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 16)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 16)        20752     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 8, 16)         0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 8, 8, 16)         

In [26]:
base_learning_rate = 1e-4
initial_epochs = 50

opt = keras.optimizers.Adam(learning_rate=base_learning_rate)

In [27]:
# Compile model
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])

In [28]:
history = model.fit(X_train, y_train, epochs = initial_epochs, validation_data=(X_test, y_test))

Epoch 1/50
4/4 [==============================] - 2s 222ms/step - loss: 0.6953 - binary_accuracy: 0.4779 - val_loss: 0.6811 - val_binary_accuracy: 0.5517
Epoch 2/50
4/4 [==============================] - 1s 155ms/step - loss: 0.6934 - binary_accuracy: 0.4956 - val_loss: 0.6854 - val_binary_accuracy: 0.6552
Epoch 3/50
4/4 [==============================] - 1s 151ms/step - loss: 0.6882 - binary_accuracy: 0.5487 - val_loss: 0.6884 - val_binary_accuracy: 0.5862
Epoch 4/50
4/4 [==============================] - 1s 153ms/step - loss: 0.6935 - binary_accuracy: 0.4956 - val_loss: 0.6889 - val_binary_accuracy: 0.5517
Epoch 5/50
4/4 [==============================] - 1s 149ms/step - loss: 0.6839 - binary_accuracy: 0.5487 - val_loss: 0.6893 - val_binary_accuracy: 0.5517
Epoch 6/50
4/4 [==============================] - 1s 148ms/step - loss: 0.6804 - binary_accuracy: 0.5664 - val_loss: 0.6875 - val_binary_accuracy: 0.5862
Epoch 7/50
4/4 [==============================] - 1s 154ms/step - loss: 0.68